# Zapytania złożone

Przed rozpoczęciem pracy konieczne jest skonfigurowanie bazy oraz załadowanie danych:
```ruby
$:.unshift "."
require 'db_setup'
require 'db_load'
```

In [1]:
$:.unshift "."
require 'db_setup'
require 'db_load'


==  AddIndicesMigration: migrating ============================================
==  AddIndicesMigration: migrated (0.0000s) ===================================

.................................................................................................

true

W dalszych zadaniach będzie wykonywać polecenia korzystając z następujących klas, zmapowanych na odpowiadające im 
table w bazie danych:
```ruby
class Author < ActiveRecord::Base
  # name      (string)
  # surname   (string)
  # born      (datetime) 
  # died      (datetime)
  # image_url (string)
  
  has_many :books
end

class Book < ActiveRecord::Base
  # title     (string)
  # author    (Author)
  # published (integer)

  belongs_to :author
end
```

ActiveRecord udostępnia interfejs pozwalający wyszukiwać dane w bazie również z wykorzystaniem związków zdefiniowanych pomiędzy
klasami. Do definiowania tych związków służą polecenia `has_many`, `belongs_to`, `has_one` oraz `has_and_belongs_to_many`.
Przykładowo jeśli chcemy dowiedzieć się kto był autorem pierwszej książki wykonujemy zapytanie:
```ruby
book = Book.first
puts book.title
puts book.author
```

In [2]:
book = Book.first
puts book.title
puts book.author


Végjáték (regény)
#<Author:0x000000031fe250>


Niestety prezentowany wynik nie jest zbyt użyteczny. Ponieważ jednak metoda `author` zwraca obiekt, możemy na nim wywoływać 
metody - tak samo jak w przypadku autora załadowanego bezpośrednio z wykorzystaniem metod zdefiniowanych dla klasy `Author`
```ruby
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"
```

In [3]:
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"


Végjáték (regény)
Orson Scott Card


Co więcej, jeśli mamy do czynienia ze związkiem jeden-do-wiele bądź wiele-do-wiele, możemy wywyływać wszystkie dotychczas poznane 
metody na kolekcji powiązanych obiektów. Istotnie ułatwia to wyszukiwanie potrzebnych danych:
```ruby
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end
```

In [4]:
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end


Anna Karenina
Azazel (powieść)
Anna Karenina


[#<Book id: 33, title: "Anna Karenina", language: nil, author_id: 35, published: 1877, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

Różnica pomiędzy tymi zapytaniami polega na tym, że w pierwszym przypadku szukamy książki, których tytuł zaczyna się na "A" 
w całym zbiorze książek, a w drugi przypadku wśród książek napisanych przez Tołstoja.

Aby umożliwić formatowanie wyników zapytania, IRuby udostępnia metodę `IRuby.html`, która pozwala na wyświetlanie kodu
HTML. Przykładowo
```ruby
IRuby.html "<h1>Test HTML-a</h1>"
```

In [5]:
IRuby.html "<h1>Test HTML-a</h1>"


"<h1>Test HTML-a</h1>"

Dzięki temu będziemy mogli formatować wyniki zapytań, a wszczególności wyświetlać obrazy przedstwiające autorów oraz książki. 
Należy jednak wziąć pod uwagę, że nie wszystkie książki oraz nie wszyscy autorzy posiadają odpowiadające im obrazki.
Aby wyświetlić imię, nazwisko oraz obraz pierwszego autora piszemy:
```ruby
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
```

In [6]:
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"


"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

Możemy również zdefniować funkcję, np. `display_author`, której zadaniem będzie wyświetlenie autora w tej postaci. 
Jedyny problem polega na tym, że funkcja `IRuby.html` może być wywoływana tylko raz. Dlatego jeśli chcemy wyświetlić kilku autorów,
musimy połączyć odpowiadające im kody HTML i wynik przekazać do funkcji `html`
```ruby
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document
```

In [7]:
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document


"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><h1>Andrzej Sapkowski</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/>"

## Zadania

### Zadanie 1

Wyświetl wszystkich autorów, którzy urodzili się przed rokiem 1900.

In [15]:
authors = Author.where(born: (Date.parse("0000-1-1"))..Date.parse("1900-1-1"))
authors.each do |author|
  puts author.name + " " + author.surname + ", " + "born: " + author.born.to_s
end

Adam Mickiewicz, born: 1798-12-24
Eliza Orzeszkowa, born: 1841-06-06
Juliusz Słowacki, born: 1809-06-06
Henryk Sienkiewicz, born: 1846-06-06
Jack London, born: 1876-01-12
Mikołaj Kopernik, born: 1473-02-19
Maria Franciszka Kozłowska, born: 1862-05-27
William Szekspir, born: 1564-04-23
John Ronald Reuel Tolkien, born: 1892-01-03
Aleksander Dumas (ojciec), born: 1802-07-24
Tadeusz Dołęga-Mostowicz, born: 1898-08-10
Vladimir Nabokov, born: 1899-04-22
Adolf Hitler, born: 1889-04-20
Benedykt Chmielowski, born: 1700-03-20
Aldous Huxley, born: 1894-07-26
Honoré de Balzac, born: 1799-05-20
Franz Kafka, born: 1883-07-03
Howard Phillips Lovecraft, born: 1890-08-20
Lew Tołstoj, born: 1828-09-09
Ilja Ilf, born: 1897-10-15
Gabriela Zapolska, born: 1857-03-30
Clive Staples Lewis, born: 1898-11-29
Iwan Turgieniew, born: 1818-11-09
Lucy Maud Montgomery, born: 1874-11-30
Aleksander Majkowski, born: 1876-07-17


[#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 3, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-11-18", image_url: nil>, #<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>, #<Author id: 5, name: "Henryk", surname: "Sienkiewicz", born: "1846-06-06", died: "1916-11-18", image_url: nil>, #<Author id: 13, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 17, name: "Mikołaj", surname: "Kopernik", born: "1473-02-19", died: "1543-05-24", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 18, name: "Maria Franciszka", surname: "Kozłowska", born: "1862-05-27", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 20, name: "William", surname: "Szekspir", born: "1564

### Zadanie 2

Zmodyfikuj funkcję wyświetlającą autorów tak aby obejmowała ich datę urodzin i śmierci w formacie `dzień.miesiąc.rok`. Następnie wyświetl wszystkich autorów,
którzy zmarli w XX wieku.

In [22]:
authors = Author.where(died: (Date.parse("1900-1-1"))..Date.parse("1999-12-31"))
authors.each do |author|
  puts author.name + " " + author.surname + ", " + "born: " + author.born.strftime("%d-%m-%Y") + ", died: " + author.died.strftime("%d-%m-%Y")
end

Eliza Orzeszkowa, born: 06-06-1841, died: 18-11-1910
Henryk Sienkiewicz, born: 06-06-1846, died: 18-11-1916
Jack London, born: 12-01-1876, died: 22-11-1916
Antoni Gołubiew, born: 25-02-1907, died: 27-06-1979
George Orwell, born: 25-06-1903, died: 21-01-1950
John Ronald Reuel Tolkien, born: 03-01-1892, died: 02-09-1973
Tadeusz Dołęga-Mostowicz, born: 10-08-1898, died: 20-09-1939
Vladimir Nabokov, born: 22-04-1899, died: 02-07-1977
Adolf Hitler, born: 20-04-1889, died: 30-04-1945
Aldous Huxley, born: 26-07-1894, died: 22-11-1963
Franz Kafka, born: 03-07-1883, died: 03-06-1924
Howard Phillips Lovecraft, born: 20-08-1890, died: 15-03-1937
Karol Olgierd Borchardt, born: 25-03-1905, died: 20-05-1986
Lew Tołstoj, born: 09-09-1828, died: 20-11-1910
Janusz Zajdel, born: 15-08-1938, died: 19-07-1985
Isaac Asimov, born: 04-10-1919, died: 06-04-1992
Ilja Ilf, born: 15-10-1897, died: 13-04-1937
Jewgienij Pietrow (pisarz), born: 13-12-1903, died: 02-07-1942
Gabriela Zapolska, born: 30-03-1857, died:

[#<Author id: 3, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-11-18", image_url: nil>, #<Author id: 5, name: "Henryk", surname: "Sienkiewicz", born: "1846-06-06", died: "1916-11-18", image_url: nil>, #<Author id: 13, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 16, name: "Antoni", surname: "Gołubiew", born: "1907-02-25", died: "1979-06-27", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 19, name: "George", surname: "Orwell", born: "1903-06-25", died: "1950-01-21", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 22, name: "John Ronald Reuel", surname: "Tolkien", born: "1892-01-03", died: "1973-09-02", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 24, name: "Tadeusz", surname: "Dołęga-Mostowicz", born: "1898-08-10", died: "1939-09-20", image_url: "http

### Zadanie 3

Wyświetl 5 najmłodszych autorów, którzy posiadają przypisane zdjęcie.

In [47]:
def display_author(author)
  "<h1>#{author.name} #{author.surname}, born: #{author.born} </h1><img src='#{author.image_url}'/>"
end
authors = Author.where("image_url is not null").order(born: :desc).limit(5)

doc1 = ""

authors.each do |author|
  doc1 << display_author(author)
end

IRuby.html doc1


"<h1>Andrzej Pilipiuk, born: 1974-03-20 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Andrzej_Pilipiuk-Polcon2006.jpg?width=300'/><h1>J. K. Rowling, born: 1965-07-31 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/J._K._Rowling_2010.jpg?width=300'/><h1>Rafał A. Ziemkiewicz, born: 1964-09-13 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Rafał_Ziemkiewicz.jpg?width=300'/><h1>Boris Akunin, born: 1956-05-20 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Boris_Akunin_2012.jpg?width=300'/><h1>Orson Scott Card, born: 1951-08-24 </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

### Zadanie 4

Zdefiniuj funkcję `display_book`, która będzie zwracała fragment kodu HTML obejmujący tytuł książki, rok jej wydania oraz
przypisany obrazek.
Wyświetl najwcześniej wydaną książkę.

In [49]:
def display_book(book)
  "<h1>#{book.title} #{book.published}</h1><img src='#{book.image_url}'/>"
end

doc2 = " "
book = Book.where("image_url is not null").order(:published).first
doc2 << display_book(book)

IRuby.html doc2

" <h1>Hamlet </h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hamlet_quarto_3rd.jpg?width=300'/>"

### Zadanie 5

Wyświetl wszystkie książki, które nie mają przypisanego roku wydania.

In [61]:
def display_book(book)
  "<h1>#{book.title} #{book.published}</h1>"
end

books = Book.where("published is null")
doc3 = ""

books.each do |book|
  doc3 << display_book(book)
end

IRuby.html doc3




"<h1>Végjáték (regény) </h1><h1>Hamlet </h1>"

### Zadanie 6

Zmodyfikuj funkcję `display_author` tak, aby wyświetlała również wszystkie książki napisane przez autora. Użyj znacznika `<ul>`.
Korzystając z tej funkcji wyświetl opis Stanisława Lema.

In [68]:
def display_author(author)
  doc = "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
  doc << "<ul>Books:"
  author.books.each do |book|
   doc << "<li>#{book.title}</li>"
  end
  doc << "</ul>"
end

author = Author.where(surname: "Lem").first
doc = display_author(author)

IRuby.html doc



"<h1>Stanisław Lem</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><ul>Books:<li>Bajki robotów</li><li>Solaris (powieść)</li><li>Bezsenność (zbiór opowiadań)</li><li>Głos Pana</li><li>Inwazja z Aldebarana (zbiór opowiadań)</li><li>Katar (powieść)</li><li>Księga robotów</li><li>Maska (zbiór dzieł)</li><li>Niezwyciężony i inne opowiadania</li><li>Noc księżycowa</li><li>Pamiętnik znaleziony w wannie</li><li>Polowanie (zbiór opowiadań)</li><li>Powtórka (zbiór dzieł)</li><li>Pożytek ze smoka</li><li>Rasa drapieżców. Teksty ostatnie</li><li>Sezam i inne opowiadania</li><li>Summa technologiae</li><li>Wizja lokalna (powieść)</li></ul>"

### Zadanie 7

Wśród książek napisanych przez Lema wyświetl wszystkie, które zawieraja w tytule słowo "zbiór".

In [83]:
def display_author(author)
  doc = "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
  doc << "<ul>Books:"
  author.books.each do |book|
    if book.title =~ /zbiór/
      doc << "<li>#{book.title}</li>"
    end
  end
  doc << "</ul>"
end

author = Author.where(surname: "Lem").first
doc = display_author(author)

IRuby.html doc



"<h1>Stanisław Lem</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><ul>Books:<li>Bezsenność (zbiór opowiadań)</li><li>Inwazja z Aldebarana (zbiór opowiadań)</li><li>Maska (zbiór dzieł)</li><li>Polowanie (zbiór opowiadań)</li><li>Powtórka (zbiór dzieł)</li></ul>"

### Zadanie 8

Wśród książek napisanych przez Lema wyświetl te, które zostały wydane w roku jego śmierci.

In [74]:
def display_author(author)
  doc = "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
  doc << "<ul>Books:"
  author.books.each do |book|
    if book.published == author.died.year
      doc << "<li>#{book.title}</li>"
    end
  end
  doc << "</ul>"
end

author = Author.where(surname: "Lem").first

doc = display_author(author)

IRuby.html doc



"<h1>Stanisław Lem</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Stanislaw_Lem_2.jpg?width=300'/><ul>Books:<li>Rasa drapieżców. Teksty ostatnie</li></ul>"

### Zadanie 9

Wyświetl wszystkie książki znajdujące się w bazie. Opis powinien zawierać tytuł, imię i nazwisko autora, rok wydania oraz 
obraz przedstawiający książkę (o ile istnieje).

In [91]:
doc = ""

books = Book.all
books.each do |book|
  doc << "<h1>#{book.title}</h1><h2>#{book.author.name} #{book.author.surname}, #{book.published}</h2><img src='#{book.image_url}'/>"
end


IRuby.html doc

"<h1>Végjáték (regény)</h1><h2>Orson Scott Card, </h2><img src=''/><h1>Pani Jeziora (powieść)</h1><h2>Andrzej Sapkowski, 1999</h2><img src=''/><h1>Bellew Zawierucha</h1><h2>Jack London, 1912</h2><img src=''/><h1>Fałszywy trop (powieść)</h1><h2>Henning Mankell, 1995</h2><img src=''/><h1>Bajki robotów</h1><h2>Stanisław Lem, 1964</h2><img src=''/><h1>Bolesław Chrobry (powieść)</h1><h2>Antoni Gołubiew, 1947</h2><img src=''/><h1>De revolutionibus orbium coelestium</h1><h2>Mikołaj Kopernik, 1543</h2><img src=''/><h1>Dzieło Wielkiego Miłosierdzia</h1><h2>Maria Franciszka Kozłowska, 1922</h2><img src=''/><h1>Folwark zwierzęcy</h1><h2>George Orwell, 1945</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Animalism_flag.svg?width=300'/><h1>Hamlet</h1><h2>William Szekspir, </h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hamlet_quarto_3rd.jpg?width=300'/><h1>Harry Potter</h1><h2>J. K. Rowling, 1997</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Harry_Potter_wordmark.svg?width=300'/><h1>Hobbit, czyli tam i z powrotem</h1><h2>John Ronald Reuel Tolkien, 1937</h2><img src=''/><h1>Hrabia Monte Christo</h1><h2>Aleksander Dumas (ojciec), 1844</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Louis_Français-Dantès_sur_son_rocher.jpg?width=300'/><h1>Kariera Nikodema Dyzmy</h1><h2>Tadeusz Dołęga-Mostowicz, 1932</h2><img src=''/><h1>Krzyżacy (powieść)</h1><h2>Henryk Sienkiewicz, 1900</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/PL_Henryk_Sienkiewicz-Krzyżacy_0005.jpeg?width=300'/><h1>Lolita</h1><h2>Vladimir Nabokov, 1955</h2><img src=''/><h1>Mein Kampf</h1><h2>Adolf Hitler, 1925</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Landsberg.Jail.jpg?width=300'/><h1>Nad Niemnem</h1><h2>Eliza Orzeszkowa, 1888</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nad_Niemnem_-_manuscript.jpg?width=300'/><h1>Niedokończone opowieści</h1><h2>Christopher Tolkien, 1980</h2><img src=''/><h1>Nowe Ateny</h1><h2>Benedykt Chmielowski, 1745</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nowe_ateny1.jpg?width=300'/><h1>Nowy wspaniały świat</h1><h2>Aldous Huxley, 1932</h2><img src=''/><h1>Ogniem i mieczem</h1><h2>Henryk Sienkiewicz, 1884</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Podbipieta.JPG?width=300'/><h1>Ojciec Goriot</h1><h2>Honoré de Balzac, 1835</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Father_Goriot_by_H._Daumier_(1842).JPG?width=300'/><h1>Proces (powieść)</h1><h2>Franz Kafka, 1925</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kafka_Der_Prozess_1925.jpg?width=300'/><h1>Psałterz Dawidów</h1><h2>Jan Kochanowski, 1579</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kochanowski_-_Psałterz_Dawidów_(1579).jpg?width=300'/><h1>Quo vadis</h1><h2>Henryk Sienkiewicz, 1896</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Rzym_popiersie_Sienkiewicza_w_kosciele_Domine_Quo_Vadis.JPG?width=300'/><h1>Romeo i Julia</h1><h2>William Szekspir, 1597</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Romeo_and_juliet_brown.jpg?width=300'/><h1>Silmarillion</h1><h2>John Ronald Reuel Tolkien, 1977</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Silmarrillion,_Just_under_the_Cover.jpg?width=300'/><h1>Solaris (powieść)</h1><h2>Stanisław Lem, 1961</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Solaris,_various_editions_02.jpg?width=300'/><h1>Władca Pierścieni</h1><h2>John Ronald Reuel Tolkien, 1954</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Unico_Anello.png?width=300'/><h1>Zew Cthulhu (opowiadanie)</h1><h2>Howard Phillips Lovecraft, 1928</h2><img src=''/><h1>Znaczy Kapitan</h1><h2>Karol Olgierd Borchardt, 1960</h2><img src=''/><h1>Anna Karenina</h1><h2>Lew Tołstoj, 1877</h2><img src='http://commons.wikimedia.org/wiki/Special:FilePath/AnnaKareninaTitle.jpg?width=300'/><h1>Azazel (powieść)</h1><h2

### Zadanie 10

Wyświetl wszystkich autorów znajdujących się w bazie wraz z opisem zawierającym ich zdjęcie, datę urodzin i śmierci oraz spis 
opublikowanych książek. Pamiętaj o odpowiednim sformatowaniu dat oraz o tym, że nie wszyscy autorzy mają przypisane wszystkie
informacje.

In [97]:
doc = ""
books = Book.all
authors = Author.all
authors.each do |author|
  doc << "<h1>#{author.name} #{author.surname}</h1>
<h3>#{author.born.strftime("%d-%m-%Y")}, #{author.died}</h3><img src='#{author.image_url}'/>
<ul>Books:"
  books.each do |book|
    if 
      author.id == book.author_id
      doc << "<li>#{book.title}</li>"
      end
  end
  doc << "</ul>"
end


IRuby.html doc

#ŹLE DATY

NoMethodError: undefined method `strftime' for nil:NilClass